In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil

#### Read the files directly from S3

In [115]:
%%time
s3 = boto3.client("s3")
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF = pd.read_csv(obj['Body'], compression = 'gzip', dtype = dtypes)

<timed exec>:25: DtypeWarning: Columns (10,15,16,19,20,47,51,52,55) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 6min 18s
Wall time: 6min 20s


In [118]:
eventsSF.tripId.unique()

array([nan, 'default'], dtype=object)

In [3]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [5]:
# Adding scenario info
eventsSF['scenario'] = "ridehail"
eventsSF['scenario'] = eventsSF['scenario'].astype("category")
eventsSF['lever'] = "price"
eventsSF['lever'] = eventsSF['lever'].astype("category")
eventsSF['year'] = 2018
eventsSF['lever_position'] = 0.5

In [6]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [7]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [8]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [9]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [10]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [11]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [12]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [13]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

CPU times: total: 1min 47s
Wall time: 1min 47s


In [14]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [15]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [16]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [17]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [20]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

CPU times: total: 4min 32s
Wall time: 4min 32s


In [21]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [22]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [23]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [24]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

CPU times: total: 17.6 s
Wall time: 17.6 s


In [25]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [26]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [27]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [28]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [29]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [30]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [31]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [32]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [33]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [34]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding the census blocks

In [35]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [36]:
# census_2010 = "https://beam-core-act.s3.amazonaws.com/deepDive/RawData/Census_Blocks/Scenario_2010_shp/"
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [37]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 58min 34s
Wall time: 58min 34s


In [38]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 59min 24s
Wall time: 1h 7s


#### Adding new columns

In [39]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [40]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [41]:
eventsSF['duration_travelling'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [42]:
eventsSF['distance_travelling'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [43]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [44]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_travelling'], np.nan)

In [45]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_travelling'], np.nan)

In [46]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_travelling'], np.nan)

In [47]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_travelling'], np.nan)

In [48]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_travelling'], np.nan)

In [49]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [50]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_travelling'], np.nan)

In [51]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_travelling'], np.nan)

In [52]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_travelling'], np.nan)

In [53]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [54]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [55]:
#eventsSF = eventsSF.set_index('IDMerged')

In [56]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [57]:
eventsSF['emissionFood'] = eventsSF['fuelFood'] * 8.3141841e-9 * 0

In [58]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [59]:
eventsSF['emissionElectricity'] = eventsSF['fuelElectricity'] * 2.77778e-10 * 947.2 * 0.0005

In [60]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [61]:
eventsSF['emissionDiesel'] = eventsSF['fuelDiesel'] * 8.3141841e-9 * 10.180e-3

In [62]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [63]:
eventsSF['emissionBiodiesel'] = eventsSF['fuelBiodiesel'] * 8.3141841e-9 * 10.180e-3

In [64]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [65]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [66]:
eventsSF['emissionGasoline'] = eventsSF['fuelGasoline'] * 8.3141841e-9 * 8.89e-3

In [67]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [68]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [69]:
# Marginal emission
conditions1  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0), 
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelGasoline'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelBiodiesel'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelDiesel'].notna() != 0),             
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),

               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelElectricity'].notna() != 0),
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelGasoline'].notna() != 0),           
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelBiodiesel'].notna() != 0),   
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelDiesel'].notna() != 0),

               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelFood'].notna() != 0)]

choices1 = [eventsSF['emissionElectricity']/eventsSF['numPassengers'],
           eventsSF['emissionGasoline']/eventsSF['numPassengers'],
           eventsSF['emissionBiodiesel']/eventsSF['numPassengers'],
           eventsSF['emissionDiesel']/eventsSF['numPassengers'],           
           0 , 
           eventsSF['emissionFood'], 
           eventsSF['emissionElectricity'],
           eventsSF['emissionGasoline'],
           eventsSF['emissionBiodiesel'],
           eventsSF['emissionDiesel'],
           eventsSF['emissionFood']]

In [70]:
eventsSF['emission_marginal'] = np.select(conditions1, choices1, default=np.nan)

In [71]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [72]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [73]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Mode Choice planned and actual

In [74]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

CPU times: total: 25.8 s
Wall time: 26 s


In [75]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

CPU times: total: 26.1 s
Wall time: 26.1 s


In [76]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [77]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [78]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [79]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [80]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

CPU times: total: 1.8 s
Wall time: 1.8 s


In [81]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [82]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [83]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 
            'actEndTime': np.sum, 
            'duration_travelling': np.sum, 
            'cost_BEAM': np.sum, 
            'actStartType': np.sum, 
            'actEndType': np.sum, 
            'duration_walking': np.sum, 
            'duration_in_privateCar': np.sum, 
            'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 
            'distance_travelling': np.sum, 
            'duration_in_transit': np.sum, 
            'distance_walking': np.sum, 
            'distance_bike': np.sum, 
            'distance_ridehail': np.sum, 
            'distance_privateCar': np.sum, 
            'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 
            'transit_subway': np.sum, 
            'transit_tram': np.sum, 
            'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 
            'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'lever_position': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 
            'fuelElectricity': np.sum, 
            'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 
            'fuel_not_Food': np.sum, 
            'fuelGasoline': np.sum, 
            'fuel_marginal': np.sum,
            'BlockGroupStart': 'first',
            'BlockGroupEnd': 'last',
            'emissionFood': np.sum, 
            'emissionElectricity': np.sum, 
            'emissionDiesel': np.sum, 
            'emissionGasoline': np.sum,
            'emissionBiodiesel': np.sum, 
            'emission_marginal': np.sum,
            'lever': lambda x: ', '.join(set(x.dropna().astype(str)))
           }).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

CPU times: total: 14min 3s
Wall time: 14min 4s


In [84]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [85]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_travelling'] 

In [86]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [87]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [88]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [89]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [90]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default=np.nan)

In [91]:
# Column with six summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car', 'ride_hail_transit']

In [92]:
Person_Trip_eventsSF['mode_choice_actual_6'] = np.select(conditions, choices, default=np.nan)

In [93]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [94]:
#Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base_core_act.csv', index = False)

#### Merging with ActivitySim files

In [95]:
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/activitysim/year-2018-iteration-5/households.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
households = pd.read_csv(obj['Body'], compression = 'gzip')

In [96]:
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/activitysim/year-2018-iteration-5/persons.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
persons = pd.read_csv(obj['Body'], compression = 'gzip')

In [97]:
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/activitysim/year-2018-iteration-5/final_tours.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
tours = pd.read_csv(obj['Body'], compression = 'gzip')

In [98]:
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/activitysim/year-2018-iteration-5/final_trips.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
trips = pd.read_csv(obj['Body'], compression = 'gzip')

In [20]:
#actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
#actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/activitysim/"
#actloc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220529/activitysim/"
#actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220529/activitysim/"
#actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
#actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

#households = pd.read_csv(actloc_2018_baseline + 'final_households.csv')
#persons = pd.read_csv(actloc_2018_baseline + 'final_persons.csv')
#tours = pd.read_csv(actloc_2018_baseline +'final_tours.csv')
#plans = pd.read_csv(actloc_2018_baseline +'final_plans.csv')
#trips = pd.read_csv(actloc_2018_baseline + 'final_trips.csv')
#landuse = pd.read_csv(actloc_2018_baseline + 'final_land_use.csv')

In [21]:
# PUMS data for Disability information
#filename = "C:/Shared-Work/Data/Disability_PUMS/custom_mpo_06197001_model_data.h5"

#with h5py.File(filename, "r") as f:
    # List all groups
    #print("Keys: %s" % f.keys())
    #a_group_key = list(f.keys())[0]

    # Get the data
    #data = list(f[a_group_key])

In [22]:
# PUMS data for Disability information
#person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')
#housing_unit_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_hca/ss13hca.csv')

In [23]:
# PUMS data for Disability information
key = "deepDive/RawData/Disability/ss13pca.csv"
obj = s3.get_object(Bucket="beam-core-act", Key=key)
person_5_year_2010 = pd.read_csv(obj['Body'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_7684\3606722351.py:4: DtypeWarning: Columns (107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  person_5_year_2010 = pd.read_csv(obj['Body'])


In [24]:
# PUMS data for Disability information
key = "deepDive/RawData/Disability/ss13hca.csv"
obj = s3.get_object(Bucket="beam-core-act", Key=key)
housing_unit_5_year_2010 = pd.read_csv(obj['Body'])

In [99]:
# Merge PUMS households and persons 
person_5_year_2010 = person_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
housing_unit_5_year_2010 = housing_unit_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
hhpersons_PUMS2010 = pd.merge(left=person_5_year_2010, right=housing_unit_5_year_2010, how='left', on='SERIALNO')

NameError: name 'person_5_year_2010' is not defined

In [100]:
# Merge BEAM households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [27]:
# Merge Disability Columns
hhpersons_PUMS2010 = hhpersons_PUMS2010.sort_values(by=['SERIALNO', 'SPORDER']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['serialno', 'PNUM']).reset_index(drop=True)
hhpersonsDIS = pd.merge(hhpersons, hhpersons_PUMS2010[['SERIALNO', 'SPORDER', 'AGEP', 'SEX', 'DIS', 'HINCP', 'VEH', 'JWMNP', 'JWRIP', 'JWTR', 'RAC1P', 'RAC2P05']], how='left',left_on = ['serialno', 'PNUM'] , right_on=['SERIALNO', 'SPORDER'])

In [102]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['person_id']).reset_index(drop=True)
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [103]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)
hhperTours = hhperTours.sort_values(by=['person_id','tour_id']).reset_index(drop=True)
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
# Merge trips, tours, households, and persons with land-use
#landuse = landuse.sort_values(by=['TAZ'])
#tourTripsMerged = tourTripsMerged.sort_values(by=['TAZ'])
#landuseTripsMerged = pd.merge(tourTripsMerged, landuse, how='left', left_on='TAZ', right_on='TAZ', suffixes=('', '_drop'))
#landuseTripsMerged.drop([col for col in landuseTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
# Concat mode_choice_raw files
#path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
#all_files = glob.glob(path + "*raw.csv")
#li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
#li2 = list(li_mapper)
#SFmode_choice_raw = pd.concat(li2, axis = 0, ignore_index=True)

In [104]:
# Concat mode_choice_utilities files - Download File from s3 bucket 
key = "pilates-outputs/sfbay-rhprice-0.5-20220814/activitysim/year-2018-iteration-5/trip_mode_choice.zip"  #the path should be updated
s3.download_file(
    Bucket="beam-outputs", Key=key, Filename="trip_mode_choice.zip")

In [105]:
# Automatically extracting ZipFile with a python command 
with ZipFile('trip_mode_choice.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [106]:
# Concat mode_choice_utilities files
path = "C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice/" #the path should be updated
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [ ]:
# Merge mode_choice_raw and utilities
#SFmode_choice_raw = SFmode_choice_raw.sort_values(by=['trip_id'])
#SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
#rawUtil = pd.merge(left=SFmode_choice_raw, right=SFmode_choice_utilities, how='left', on='trip_id')

In [108]:
#just utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=tourTripsMerged, right=SFmode_choice_utilities, how='left', on=['trip_id'])

In [114]:
Person_Trip_eventsSF.head()

,IDMerged,tripIndex,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6
0,1,default,060855033222,060855035091,114165.000000,Homeothmaintschool,115133.000000,othmaintschoolHome,0.000000,0.000000,18593.944000,18593.944000,0.000000,0.000000,18593.944000,0.000000,968.000000,0.000000,0.000000,0.000000,968.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"body-1,teleportationSharedVehicle-1-1,body-1bo...",price,0.5,hov2_teleportation,hov2_teleportation,,,,0,0,ridehail,0,0,0,0,0,,2018,968.000000,0.000000,Homeothmaintschool_to_othmaintschoolHome,car,car
1,2,default,060855033222,060855033222,245734.000000,Homeworkatworkworkothmaint,258749.000000,workatworkworkothmaintHome,10.318477,0.000000,101393.897000,97987.504000,0.000000,0.000000,109571.858000,11584.354000,4105.000000,0.000000,0.000000,0.000000,13015.000000,8910.000000,0.000000,0.000000,0.000000,0.000000,0.011117,0.000000,0.000000,0.000000,0.000000,613970.762000,150411373.977745,151025344.739745,150411373.977745,"body-2,543097,543097,body-2body-2,543097,54309...",price,0.5,car,car,"1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0...","Public, Public, Public, Public, Public, Public...",,0,0,ridehail,0,0,0,0,0,"body-2, 543097",2018,13015.000000,0.000000,Homeworkatworkworkothmaint_to_workatworkworkot...,car,car
2,3,default,060855033222,060855035091,217452.000000,HomeworkHomeworkwork,222141.000000,workHomeworkworkHome,6.160751,0.000000,58224.585000,56771.790000,0.000000,0.000000,59369.078000,2597.288000,2690.000000,0.000000,0.000000,0.000000,4689.000000,1999.000000,0.000000,0.000000,0.000000,0.000000,0.006262,0.000000,0.000000,0.000000,0.000000,137656.264000,84724582.975524,84862239.239524,84724582.975524,"body-3,543098,543098,body-3body-3,543098,54309...",price,0.5,car,car,"1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0...","Public, Workplace, Workplace, Residential, Res...",,0,0,ridehail,0,0,0,0,0,"543098, body-3",2018,4689.000000,0.000000,HomeworkHomeworkwork_to_workHomeworkworkHome,car,car
3,4,default,060855033222,060855035091,92119.000000,Homeschool,93070.000000,schoolHome,0.000000,0.000000,17265.772000,17265.772000,0.000000,0.000000,17265.772000,0.000000,951.000000,0.000000,0.000000,0.000000,951.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"body-4,teleportationSharedVehicle-1-4,body-4bo...",price,0.5,hov3_teleportation,hov3_teleportation,,,,0,0,ridehail,0,0,0,0,0,,2018,951.000000,0.000000,Homeschool_to_schoolHome,car,car
4,86,default,060816070002,060816070002,250546.000000,HomeothdiscrHomeshoppingothmaint,253932.000000,othdiscrHomeshoppingothmaintHome,3.601521,0.000000,34127.231000,32836.807000,0.000000,0.000000,34127.231000,1290.424000,1440.000000,0.000000,0.000000,0.000000,3386.000000,1946.000000,0.000000,0.000000,0.000000,0.000000,0.004402,0.000000,0.000000,0.000000,0.000000,68392.472000,59553628.763097,59622021.235097,59553628.763097,"body-86,730956,730956,body-86body-86,730956,73...",price,0.5,car,car,"1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0...","Public, Public, Public, Residential, Residenti...",,0,0,rid

In [113]:
SFActMerged.head()

,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,577,1,1735309,72,school,1,True,2,othmaint,583,641,5.883635,8.000000,SHARED2FREE,-1.900483,school,1,1,1,1,mandatory,1,633.000000,641.000000,1735309,61.000000,8.000000,15.000000,7.000000,NaN,NaN,SHARED2PAY,-0.968793,NaN,NaN,1out_0in,school,0.000000,0,1,0,male,3,1,1735309,0.000000,5,19 and under,1.000000,633,6,asian,0,3,-1,no,2,0,641,8,4,1,-121.806440,37.350304,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1,False,641,19.798008,633,2.572545,24.576923,-1,NaN,NaN,False,NaN,0.000000,0.000000,0.000000,False,M,True,False,False,False,school1,False,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2,6085,yes,60855035091006,1,no,35,6,own not recent,four or more,yes,1,two or more,240000.000000,gt35-lt65,2010000482865,2.000000,asian,gt150,0,two or more,0,no,4,3,641,4,0.097000,60138,240.000000,4,12.860000,29.682171,2.000000,2,2,2,2,0,0,0,1,True,False,False,False,2,2.000000,0,4,2,2,2,2,True,0_tours,0,-999.146592,-999.147009,-2.079126,-2.079364,-999.145240,-999.145407,-12.670279,-1001.088371,-1000.459856,-1998.770467,-1998.770467,-1998.770467,-1998.770467,-1999.307149,-2997.409495,-2997.409495,-2997.409495,-2997.409495,-8.011878,-7.553389,-8.095143
1,578,1,1735309,72,school,2,True,2,school,633,583,NaN,8.000000,SHARED2FREE,-2.206079,school,1,1,1,1,mandatory,

In [ ]:
# Both raw and utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
#tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
#rawUtil = rawUtil.sort_values(by=['trip_id'])
#SFActMerged= pd.merge(left=tourTripsMerged, right=rawUtil, how='left', on=['trip_id'])

In [109]:
# Merge person_trip level BEAM with activity sim merged files
SFActMerged = SFActMerged.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=SFActMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [ ]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [ ]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [ ]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [160]:
# Save the output file to S3
eventsASim.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Ridehail_Price/sf_2018_base_8_10_22.csv', index=False)  #the path should be updated

CPU times: total: 6min 17s
Wall time: 8min 1s


In [46]:
# Delete the utilities files downloaded and saved in the system
os.remove('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice.zip')   #the path should be updated
shutil.rmtree('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice')   #the path should be updated